In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days


In [2]:
import ujson as json

In [3]:
import sys

In [4]:
sys.path.insert(0, '/home/bmcfee/git/laplacian_segmentation/code')

In [5]:
import segmenter
import scipy

In [14]:
#segmenter.FILTER_WIDTH = 9
#segmenter.REP_WIDTH = 5
MAX_LINKS = 3

In [6]:
def normalize_labels(labels):
    
    remap = {}
    v = 0
    
    for i in labels:
        if i in remap:
            continue
        remap[i] = v
        v = v + 1
        
    return [remap[i] for i in labels]

In [7]:
def do_segmentation(X_rep, X_loc, beats):

    #X_rep, X_loc = X
    # Find the segment boundaries
    k_min, k_max  = segmenter.get_num_segs(beats[-1, 1], segmenter.MIN_SEG, segmenter.MAX_SEG)

    # Get the raw recurrence plot
    Xpad = np.pad(X_rep, [(0,0), (segmenter.N_STEPS, 0)], mode='edge')
    Xs = librosa.feature.stack_memory(Xpad, n_steps=segmenter.N_STEPS)[:, segmenter.N_STEPS:]

    k_link = 1 + int(np.ceil(2 * np.log2(X_rep.shape[1])))
    R = librosa.segment.recurrence_matrix(  Xs, 
                                            k=k_link, 
                                            width=segmenter.REP_WIDTH, 
                                            metric=segmenter.METRIC,
                                            sym=True).astype(np.float32)
    # Get the knn links
    links = []
    for i in range(len(R)):
        links.append(list(np.flatnonzero(R[i,:])))
        if len(links[-1]) > MAX_LINKS:
            links[-1] = links[-1][:MAX_LINKS]
    
    # Generate the repetition kernel
    A_rep = segmenter.self_similarity(Xs, k=k_link)

    # And the local path kernel
    A_loc = segmenter.self_similarity(X_loc, k=k_link)

    # Mask the self-similarity matrix by recurrence
    #S = librosa.segment.recurrence_to_lag(R)

    #Sf = segmenter.clean_reps(S)
    tf = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = tf(R, size=(1, segmenter.FILTER_WIDTH))
    # De-skew
    #Rf = librosa.segment.structure_feature(Sf, inverse=True)

    # Symmetrize by force
    Rf = np.maximum(Rf, Rf.T)

    # Suppress the diagonal
    Rf[np.diag_indices_from(Rf)] = 0

    # We can jump to a random neighbor, or +- 1 step in time
    # Call it the infinite jukebox matrix
    T = segmenter.combine_graphs(Rf * A_rep, (np.eye(len(A_loc),k=1) + np.eye(len(A_loc),k=-1)) * A_loc)
    
    # Get the graph laplacian
    L = segmenter.sym_laplacian(T)

    # Get the bottom k eigenvectors of L
    Lf = segmenter.factorize(L, k=1+segmenter.MAX_REP)
    
    B, L = [], []
    
    parse = []
    for k in range(1, len(Lf)):
        boundaries, labels = segmenter.fixed_partition(Lf, k)
        parse.append({'boundaries': list(boundaries),
                      'labels':     normalize_labels(list(labels))})
        
    
    return parse, links

In [17]:
import os
import re

def get_meta(filename):
    R = re.match('.*/(?P<artist>.*)-(?P<title>.*).mp3', filename)
    #R = re.match('.*/(?P<artist>The Beatles)/.*/.*_-_(?P<title>.*).flac', filename)
    return R.groups()

In [18]:
def process_audio(infile, outfile):
    X_rep, X_loc, beats = segmenter.features(infile)
    parse, links = do_segmentation(X_rep, X_loc, beats)
    meta = get_meta(infile)
    json.dump({'filename': infile, 
               'artist': meta[0], 
               'title': meta[1],
               'beats': list(beats[:, 0]),
               'duration': float(beats[-1, 1]),
               'links': links,
               'segments': parse}, open(outfile, 'w'))

In [19]:
files = librosa.util.find_files('/home/bmcfee/data/CAL500/mp3/')
#files = librosa.util.find_files('/home/bmcfee/data/beatles_iso/audio/The Beatles/')
#files = librosa.util.find_files('//home/bmcfee/Music/IAYD - Supergalactic/', recurse=False, ext='mp3')
#files = librosa.util.find_files('//home/bmcfee/', recurse=False, ext='mp3')

In [20]:
files

['/home/bmcfee/data/CAL500/mp3/10cc-for_you_and_i.mp3',
 '/home/bmcfee/data/CAL500/mp3/2pac-trapped.mp3',
 '/home/bmcfee/data/CAL500/mp3/5th_dimension-one_less_bell_to_answer.mp3',
 '/home/bmcfee/data/CAL500/mp3/a_tribe_called_quest-bonita_applebum.mp3',
 '/home/bmcfee/data/CAL500/mp3/aaron_neville-tell_it_like_it_is.mp3',
 '/home/bmcfee/data/CAL500/mp3/abba-s.o.s..mp3',
 '/home/bmcfee/data/CAL500/mp3/abc-poison_arrow.mp3',
 '/home/bmcfee/data/CAL500/mp3/ac_dc-dirty_deeds_done_dirt_cheap.mp3',
 '/home/bmcfee/data/CAL500/mp3/adam_and_the_ants-prince_charming.mp3',
 '/home/bmcfee/data/CAL500/mp3/adam_ant-wonderful.mp3',
 '/home/bmcfee/data/CAL500/mp3/adverts-gary_gilmores_eyes.mp3',
 '/home/bmcfee/data/CAL500/mp3/aerobic_jonquil-sweat_machine.mp3',
 '/home/bmcfee/data/CAL500/mp3/aerosmith-dude_looks_like_a_lady.mp3',
 '/home/bmcfee/data/CAL500/mp3/aimee_mann-wise_up.mp3',
 '/home/bmcfee/data/CAL500/mp3/air-sexy_boy.mp3',
 '/home/bmcfee/data/CAL500/mp3/al_green-sha-la-la_make_me_happy.mp3

In [21]:
ids = range(len(files))
#ids = [0,1]
np.random.shuffle(ids)

In [ ]:
for i in ids[:30]:
    print i, files[i]
    if os.path.exists('/home/bmcfee/git/lsd_viz/data/%08d.json' % (12000 + i)):
        continue
    process_audio(files[i], '/home/bmcfee/git/lsd_viz/data/%08d.json' % (12000 + i))

448 /home/bmcfee/data/CAL500/mp3/television-venus.mp3
401 /home/bmcfee/data/CAL500/mp3/sly_and_the_family_stone-just_like_a_baby.mp3
492 /home/bmcfee/data/CAL500/mp3/wicked_boy-pressure.mp3
461 /home/bmcfee/data/CAL500/mp3/tom_paul-little_part_of_me.mp3
324 /home/bmcfee/data/CAL500/mp3/my_bloody_valentine-when_you_sleep.mp3
132 /home/bmcfee/data/CAL500/mp3/charlie_rich-behind_closed_doors.mp3
477 /home/bmcfee/data/CAL500/mp3/van_halen-aint_talkin_bout_love.mp3
387 /home/bmcfee/data/CAL500/mp3/sebadoh-soul_and_fire.mp3
111 /home/bmcfee/data/CAL500/mp3/byrds-wasnt_born_to_follow.mp3
338 /home/bmcfee/data/CAL500/mp3/norine_braun-spanish_banks.mp3
77 /home/bmcfee/data/CAL500/mp3/blur-country_house.mp3
	[1/5] loading audio
	[2/5] Separating harmonic and percussive signals
	[3/5] detecting beats
	[4/5] generating CQT
	[5/5] generating MFCC
0 /home/bmcfee/data/CAL500/mp3/10cc-for_you_and_i.mp3
	[1/5] loading audio
	[2/5] Separating harmonic and percussive signals